In [1]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

Python-dotenv could not parse statement starting at line 4


True

In [2]:
import time
import pickle
import pandas as pd
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import display, Markdown

from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index import GPTVectorStoreIndex, ResponseSynthesizer
from llama_index.data_structs.node import Node, DocumentRelationship

from langchain.chat_models import ChatOpenAI
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, ServiceContext
from llama_index import GPTListIndex
from llama_index.indices.list.retrievers import ListIndexLLMRetriever
from llama_index.indices.postprocessor import (
    LLMRerank
)
from llama_index.indices.postprocessor.node import AutoPrevNextNodePostprocessor

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.indices.postprocessor.node import PrevNextNodePostprocessor
from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.storage_context import StorageContext

from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

In [3]:
with open('/home/marshath/play/chainlink/algovate/algovate/data/combined_documents.pkl', 'rb') as f:
    cl_combineddoc = pickle.load(f)

# convert to llama documents
llama_cl_combineddoc = []
for doc in cl_combineddoc:
    llama_cl_combineddoc.append(Document(text=doc.page_content, extra_info=doc.metadata))


In [4]:
# df = pd.read_csv("/home/marshath/play/chainlink/algovate/algovate/data/more-chainlink-tests.csv", header=None)
# df.columns = ["question", "source"]
# df["answer"] = ""

In [5]:
# current_question_index = 0

# # Create text box and display the first question
# answer_box = widgets.Textarea(value=df['answer'][current_question_index], placeholder='Type your answer here', rows=3)
# question_display = widgets.HTML(value=f"<h4>Question {current_question_index + 1}: {df['question'][current_question_index]}</h4>")
# display(question_display)
# display(answer_box)

# # Function to update the DataFrame with the answer and display the next question
# def update_answer(btn):
#     global current_question_index
#     df.at[current_question_index, 'answer'] = answer_box.value  # Update the answer in the DataFrame
#     current_question_index += 1  # Move to the next question

#     if current_question_index < len(df):
#         question_display.value = f"<h4>Question {current_question_index + 1}: {df['question'][current_question_index]}</h4>"
#         answer_box.value = df['answer'][current_question_index]
#     else:
#         next_button.disabled = True
#         next_button.description = 'Answers saved!'

# # Create a button to save the answers or move to the next question
# next_button = widgets.Button(description='Next Question')
# next_button.on_click(update_answer)

# # Display the next button
# display(next_button)

In [6]:
# df.to_csv("/home/marshath/play/chainlink/algovate/algovate/data/more-chainlink-tests.csv", index=False)
# df.dropna(inplace=True)

In [7]:
def retrieve_docs(df, query_engine):
    # Add retrieved_docs column if it doesn't exist
    df['retrieved_docs'] = df.get('retrieved_docs', '')

    for index, row in df.iterrows():
        retrieved_docs = query_engine.retrieve(row['question'])

        docs = ""
        for d in retrieved_docs:
            if docs == "":
                docs = d.node.text
            else:
                docs = docs + "\n\n:::NEXT DOC:::\n\n" + d.node.text

        # Store retrieved docs in the DataFrame
        df.at[index, 'retrieved_docs'] = docs

In [8]:
def evaluate_dataframe_answer(df):
    # Add right_or_wrong column if it doesn't exist
    if 'right_or_wrong' not in df.columns:
        df['right_or_wrong'] = ''

    # Widgets
    button_right = widgets.Button(description="Right")
    button_wrong = widgets.Button(description="Wrong")
    button_partial = widgets.Button(description="partial")
    out = widgets.Output()

    # Starting index
    index = [0]  # Use list to allow mutation inside functions

    def display_row(row):
        """
        Display the row data in markdown
        """
        with out:
            display(Markdown(f"**Question**: {row['question']}"))
            display(Markdown(f"**Answer**: {row['answer']}"))
            display(Markdown(f"**Result**: {row['result']}\n"))

    def on_button_right_clicked(b):
        """
        Update right_or_wrong column with 'right' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'right'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    def on_button_wrong_clicked(b):
        """
        Update right_or_wrong column with 'wrong' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'wrong'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    def on_button_partial_clicked(b):
        """
        Update right_or_wrong column with 'partial' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'partial'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    button_right.on_click(on_button_right_clicked)
    button_wrong.on_click(on_button_wrong_clicked)
    button_partial.on_click(on_button_partial_clicked)

    display(out)
    display(button_right, button_wrong, button_partial)

    # Start by displaying the first row
    display_row(df.iloc[0])

In [9]:
def evaluate_dataframe_retrieval(df):
    # Add rd_right_or_wrong column if it doesn't exist
    df['rd_right_or_wrong'] = df.get('rd_right_or_wrong', '')

    df_wrong = df[df['right_or_wrong'].isin(['wrong', 'partial'])].copy()

    # Widgets
    button_right = widgets.Button(description="Right")
    button_wrong = widgets.Button(description="Wrong")
    button_partial = widgets.Button(description="Partial")
    out = widgets.Output()

    # Starting index
    index = [0]  # Use list to allow mutation inside functions

    def display_row(row):
        """
        Display the row data in markdown
        """
        with out:
            display(Markdown(f"**Question**: {row['question']}"))
            display(Markdown(f"**Answer**: {row['answer']}"))
            display(Markdown(f"**Result**: {row['result']}\n"))
            display(Markdown(f"**Retrieved Docs**: {row['retrieved_docs']}"))

    def on_button_right_clicked(b):
        """
        Update right_or_wrong column with 'right' and go to next row
        """
        df.at[df_wrong.index[index[0]], 'rd_right_or_wrong'] = 'right'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df_wrong):
            display_row(df_wrong.iloc[index[0]])

    def on_button_wrong_clicked(b):
        """
        Update right_or_wrong column with 'wrong' and go to next row
        """
        df.at[df_wrong.index[index[0]], 'rd_right_or_wrong'] = 'wrong'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df_wrong):
            display_row(df_wrong.iloc[index[0]])

    def on_button_partial_clicked(b):
        """
        Update right_or_wrong column with 'partial' and go to next row
        """
        df.at[df_wrong.index[index[0]], 'rd_right_or_wrong'] = 'partial'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df_wrong):
            display_row(df_wrong.iloc[index[0]])

    button_right.on_click(on_button_right_clicked)
    button_wrong.on_click(on_button_wrong_clicked)
    button_partial.on_click(on_button_partial_clicked)

    display(out)
    display(button_right, button_wrong, button_partial)

    # Start by displaying the first row
    if len(df_wrong) > 0:
        display_row(df_wrong.iloc[0])
    else:
        print("There are no wrong rows in the DataFrame.")

In [10]:
df = pd.read_csv("/home/marshath/play/chainlink/algovate/algovate/data/more-chainlink-tests.csv")

### Base

In [18]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0,))

max_input_size = 4096
num_output = 256
max_chunk_overlap = 20

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

In [19]:
index_base_combineddoc = GPTVectorStoreIndex.from_documents(
    llama_cl_combineddoc, 
    service_context=service_context
)

query_engine_base_combineddoc = index_base_combineddoc.as_query_engine()

KeyboardInterrupt: 

In [13]:
df2 = df.copy()
for i, gt in df2.iterrows():
    try:
        df2.loc[i, "result"] = query_engine_base_combineddoc.query(gt.question)
    except Exception as e:
        print(e)
        df2.loc[i, "result"] = "ERROR"

    print(f"Question {i+1} of {len(df2)}")

Question 1 of 12
Question 2 of 12
Question 3 of 12
Question 4 of 12
Question 5 of 12
Question 6 of 12
Question 7 of 12
Question 8 of 12
Question 9 of 12
Question 10 of 12
Question 11 of 12
Question 12 of 12


In [14]:
retrieve_docs(df2, query_engine_base_combineddoc)

In [15]:
evaluate_dataframe_answer(df2)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [16]:
evaluate_dataframe_retrieval(df2)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='Partial', style=ButtonStyle())

In [17]:
df2.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combined_base_altq.csv", index=False)

### LLMReranker

In [20]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)
reranker = LLMRerank(choice_batch_size=5, top_n=3, service_context=service_context)

index_llmreranker_combineddoc = GPTVectorStoreIndex.from_documents(llama_cl_combineddoc, service_context=service_context)

query_engine_llmreranker_combineddoc = index_llmreranker_combineddoc.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[reranker],
)

In [21]:
df3 = df.copy()
for i, gt in df3.iterrows():
    try:
        df3.loc[i, "result"] = query_engine_base_combineddoc.query(gt.question)
    except Exception as e:
        print(e)
        df3.loc[i, "result"] = "ERROR"

    print(f"Question {i+1} of {len(df2)}")

Question 1 of 12
Question 2 of 12
Question 3 of 12
Question 4 of 12
Question 5 of 12
Question 6 of 12
Question 7 of 12
Question 8 of 12
Question 9 of 12
Question 10 of 12
Question 11 of 12
Question 12 of 12


In [22]:
retrieve_docs(df3, query_engine_llmreranker_combineddoc)

In [23]:
evaluate_dataframe_answer(df3)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [24]:
evaluate_dataframe_retrieval(df3)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='Partial', style=ButtonStyle())

In [25]:
df3.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combined_llmreranker_altq.csv", index=False)

### AutoPrevNext

In [26]:
service_context = ServiceContext.from_defaults()

nodes = service_context.node_parser.get_nodes_from_documents(llama_cl_combineddoc)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
storage_context = StorageContext.from_defaults(docstore=docstore)

index_autopn_combineddoc = GPTVectorStoreIndex(nodes, storage_context=storage_context)

node_postprocessor = AutoPrevNextNodePostprocessor(docstore=docstore, num_nodes=4, service_context=service_context)

query_engine_autopn_combineddoc = index_autopn_combineddoc.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[node_postprocessor],
    response_mode="tree_summarize"
)

In [27]:
df4 = df.copy()
for i, gt in df4.iterrows():
    try:
        df4.loc[i, "result"] = query_engine_base_combineddoc.query(gt.question)
    except Exception as e:
        print(e)
        df4.loc[i, "result"] = "ERROR"

    print(f"Question {i+1} of {len(df2)}")

Question 1 of 12
Question 2 of 12
Question 3 of 12
Question 4 of 12
Question 5 of 12
Question 6 of 12
Question 7 of 12
Question 8 of 12
Question 9 of 12
Question 10 of 12
Question 11 of 12
Question 12 of 12


In [28]:
retrieve_docs(df4, query_engine_autopn_combineddoc)

In [29]:
evaluate_dataframe_answer(df4)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [32]:
evaluate_dataframe_retrieval(df4)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='Partial', style=ButtonStyle())

In [33]:
df4.to_csv("/home/marshath/play/chainlink/algovate/test_llama/combined_autopn_altq.csv", index=False)